In [1]:
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint
import re
import time
import itertools
from datetime import datetime
from urllib.parse import urlparse,urlsplit,parse_qs,parse_qsl,urlencode,urlunparse


# TUN AIRPORT

In [2]:
def airport_TUN(type_='departures'):
    list_ = []
    rows=[]
    regex = re.compile('[@_!#$%^&*()<>?/\|}{~]')

    if type_=='departures':
        html = requests.get("http://www.aeroport-de-tunis-carthage.com/tunisie-aeroport-de-tunis-carthage-vols-depart.php").text
    elif type_=='arrivals': 
        html = requests.get("http://www.aeroport-de-tunis-carthage.com/tunisie-aeroport-de-tunis-carthage-vols-arrivee.php").text

    soup = BeautifulSoup(html, "html.parser")
    
    i=0
    for row in soup.find_all('tr'):
        temp_cols = row.find_all('td')
        if len(temp_cols) > 1 and ":" in temp_cols[0].text and not temp_cols[0].table and row.get_text():
            rows.append(row.text)  
    for i in range(len(rows)):
        x = re.split("\n",rows[i])
        x = re.split("\n\n",rows[i])
        i=0
        x = list(filter(None, x))
        for i in range(len(x)):
            if  regex.search(x[i]) == None:
                    b = re.split("\n",x[i])
                    b = list(filter(None, b))                    
                    list_.append(b)  
    list_.sort()
    list_ = list(list_ for list_,_ in (itertools.groupby(list_))) 
    if len(list_[0])== 1:
        del list_[0]
    if list_[len(list_)-1][1] in {'Destination','Origine','Origin'}:
        list_.pop(len(list_)-1)
    list_ = [{'time':datetime.strptime(i[0],'%H:%M:%S').time(),'destination':i[1],'airline':i[2],'flight':i[3],'status':i[4]} for i in list_]
    for i in list_:
        if ":" in i.get('status',''):
            status,status_time = i.get('status').split(' ')
            i.update({'status':status,'status_time':datetime.strptime(status_time,'%H:%M').time()})
    return list_

In [3]:
x = airport_TUN(type_='arrivals')

In [4]:
x

[{'time': datetime.time(0, 20),
  'destination': 'FRANCFORT',
  'airline': 'LUFTHANSA',
  'flight': 'LH1326',
  'status': 'ATTERRI',
  'status_time': datetime.time(0, 21)},
 {'time': datetime.time(1, 40),
  'destination': 'MADRID',
  'airline': 'AIR EUROPA',
  'flight': 'UX1115',
  'status': 'ATTERRI',
  'status_time': datetime.time(1, 48)},
 {'time': datetime.time(4, 0),
  'destination': 'CONAKRY',
  'airline': 'TUNISAIR',
  'flight': 'TU841',
  'status': 'ATTERRI',
  'status_time': datetime.time(4, 34)},
 {'time': datetime.time(4, 50),
  'destination': 'NIAMEY AD',
  'airline': 'TUNISAIR',
  'flight': 'TU399',
  'status': 'ATTERRI',
  'status_time': datetime.time(4, 20)},
 {'time': datetime.time(5, 35),
  'destination': 'NOUAKCHOTT',
  'airline': 'TUNISAIR',
  'flight': 'TU564',
  'status': 'ATTERRI',
  'status_time': datetime.time(6, 7)},
 {'time': datetime.time(7, 45),
  'destination': 'JEDDAH',
  'airline': 'TUNISAIR',
  'flight': 'TU714',
  'status': 'ATTERRI',
  'status_time': d

# TUN OACA GET DETAILS WITH PARAMETRE (DEST, DATE ... PRIVATE)